In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as pch
import matplotlib.backends.backend_pdf as pdf

import eleanor_constants as EL
import A_model_diagnostics as A
import B_model_foraging_task as B
import C_model_toxin_avoidance_task as C

matplotlib.rc('text', usetex=True)
matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
matplotlib.rcParams['svg.fonttype'] = 'path'

savename = "./figures/6.pdf"

In [ ]:
# DECLARE ALL OF THE FILE NAMES WE PLAN TO ANALYZE AND VISUALIZE
n_trials = 1000

# APETITIVE TESTS
anosmic_fed = './data/simulation_results/'+str(n_trials)+'_simulations_anosmic_fed.csv'
anosmic_str = './data/simulation_results/'+str(n_trials)+'_simulations_anosmic_starved.csv'
orthokinesis_fed = './data/simulation_results/'+str(n_trials)+'_simulations_orthokinesis_fed.csv'
orthokinesis_str = './data/simulation_results/'+str(n_trials)+'_simulations_orthokinesis_starved.csv'
klinokinesis_fed = './data/simulation_results/'+str(n_trials)+'_simulations_klinokinesis_fed.csv'
klinokinesis_str = './data/simulation_results/'+str(n_trials)+'_simulations_klinokinesis_starved.csv'
chemo_1_fed = './data/simulation_results/'+str(n_trials)+'_simulations_chemotaxis_1_fed.csv'
chemo_1_str = './data/simulation_results/'+str(n_trials)+'_simulations_chemotaxis_1_starved.csv'

# AVERSIVE TESTS
anosmic_averse_fed = './data/simulation_results/'+str(n_trials)+'_simulations_anosmic_averse_fed.csv'
anosmic_averse_str = './data/simulation_results/'+str(n_trials)+'_simulations_anosmic_averse_starved.csv'
orthokinesis_averse_fed = './data/simulation_results/'+str(n_trials)+'_simulations_orthokinesis_averse_fed.csv'
orthokinesis_averse_str = './data/simulation_results/'+str(n_trials)+'_simulations_orthokinesis_averse_starved.csv'
klinokinesis_averse_fed = './data/simulation_results/'+str(n_trials)+'_simulations_klinokinesis_averse_fed.csv'
klinokinesis_averse_str = './data/simulation_results/'+str(n_trials)+'_simulations_klinokinesis_averse_starved.csv'
chemo_1_averse_str = './data/simulation_results/'+str(n_trials)+'_simulations_chemotaxis_averse_1_starved.csv'
chemo_1_averse_fed = './data/simulation_results/'+str(n_trials)+'_simulations_chemotaxis_averse_1_fed.csv'

In [ ]:
df = pd.read_csv('./data/trajectories/summary/cleaned_animal_analyses.csv')

feddata = df[(df["A_starved"] == "no")]
starveddata = df[(df["A_starved"] == "1day")]

tests = ["A_time_move", "A_time_wall"]
for test in tests:
    print(test, df[test].min(), df[test].max())

In [ ]:
fig = plt.figure(figsize=(7, 12))

grid = matplotlib.gridspec.GridSpec(8, 4)
ax1 = fig.add_subplot(grid[3:5, :], facecolor="None")
ax2 = fig.add_subplot(grid[0:2, 0:2], facecolor="None")
ax3 = fig.add_subplot(grid[0:2, 2:], facecolor="None")

colors = [EL.c_fed, EL.c_starve]
markers = ["o", "x"]
markersizes = [EL.s, EL.s+5]
datums = ["A_time_move", "A_time_wall"]
ps = ["****", "****"]

ax2.set_ylabel("Time spent moving (\%)")
ax3.set_ylabel("Time spent near wall (\%)")

for i, (datum, ax) in enumerate(zip(datums, [ax2, ax3])):
    data = [feddata[datum].values, starveddata[datum].values]
    jitter = [np.random.normal(scale=0.1, size=len(feddata)), np.random.normal(scale=0.1, size=len(starveddata))]
    parts = ax.violinplot(data, showmeans=False, showmedians=False,showextrema=False)

    for i, pc in enumerate(parts['bodies']):
        pc.set_facecolor(colors[i])
        pc.set_alpha(0.3)

    for i, (j, d) in enumerate(zip(jitter, data)):
        j = [x+i+1 for x in j]
        ax.scatter(j, d, alpha=0.5, color=colors[i], zorder=5, s=markersizes[i], 
                    marker=markers[i])

    ch = 0.03
    ax.plot([1-ch, 1+ch], [np.mean(data[0])]*2, color="k", alpha=0.5, lw=2, zorder=20)
    ax.plot([2-ch, 2+ch], [np.mean(data[1])]*2, color="k", alpha=0.5, lw=2, zorder=20)
    
    up, pchv = 1, ax.get_ylim()[1]*0.025
    ax.plot([1, 2], [up, up], color="k", lw=1, clip_on=False, alpha=0.5)
    ax.text(1.5, up+pchv, ps[i], alpha=0.5, clip_on=False, ha="center", va="center", zorder=12)
    ax.set_xticks(np.arange(1, 3, 1))
    ax.set_xticklabels(["Fed", "Starved"])
    ax.xaxis.set_ticks_position('none') 
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.set_xlim(0.4, 2+(1-0.4))
    ax.set_ylim(0, 1)
    ax.set_yticks(np.arange(0, 1, 0.1), minor=True)

fed_compare = [chemo_1_fed, orthokinesis_fed, klinokinesis_fed, anosmic_fed]
str_compare = [chemo_1_str, orthokinesis_str, klinokinesis_str, anosmic_str]
colors = [EL.c_chemo, EL.c_ortho, EL.c_klino, EL.c_anosmic]
labels = ["Chemotaxis", "Orthokinesis", "Klinokinesis", "Anosmic"]

for i, (fed, starve) in enumerate(zip(fed_compare, str_compare)):
    fed_df = pd.read_csv(fed, index_col=None)
    widths = fed_df.columns.astype(int).tolist()
    widths = [int(x) for x in widths]
    means = fed_df.mean(axis=0).tolist()
    sterr = fed_df.sem(axis=0).tolist()
    fed_x, fed_y, fed_s = [w/10 for w in widths], [m/60 for m in means], [s/60 for s in sterr]
    
    str_df = pd.read_csv(starve, index_col=None)
    widths = str_df.columns.astype(int).tolist()
    widths = [int(x) for x in widths]
    means = str_df.mean(axis=0).tolist()
    sterr = str_df.sem(axis=0).tolist()
    str_x, str_y, str_s = [w/10 for w in widths], [m/60 for m in means], [s/60 for s in sterr]
    
    ax1.scatter(fed_x, fed_y, clip_on=False, alpha=1, color=colors[i], marker="o", s=EL.s, label=labels[i])
    ax1.plot(fed_x, fed_y, clip_on=False, alpha=0.75, color=colors[i], label=None)
    ax1.errorbar(fed_x, fed_y, yerr=fed_s, xerr=0, fmt='|', alpha=0.25, color=colors[i], label=None)
    
    ax1.scatter(str_x, str_y, clip_on=False, alpha=1, color=colors[i], marker="x", s=35, label=None)
    ax1.plot(str_x, str_y, clip_on=False, alpha=0.75, color=colors[i], label=None)
    ax1.errorbar(str_x, str_y, yerr=str_s, xerr=0, fmt='|', alpha=0.25, color=colors[i], label=None)
    
    ax1.fill_between(fed_x, fed_y, str_y, color=colors[i], alpha=0.25, label=None)

ax1.set_ylabel("Discovery time (minutes)")
ax1.set_ylim(0, 90)
ax1.set_yticks(np.arange(0, 91, 30))
ax1.set_yticks(np.arange(0, 90, 10), minor=True)
ax1.set_xlim(3, 20)
ax1.set_xticks(np.arange(4, 21, 4))
ax1.set_xticks(np.arange(3, 20, 1), minor=True)
ax1.spines['right'].set_visible(False)
ax1.spines['top'].set_visible(False)
ax1.set_xlabel("Arena width (cm)")
for label, size in zip(EL.key_labels, EL.key_sizes): 
    ytop = ax1.get_ylim()[1]
    ax1.plot([size, size], [0, ytop], color='k', alpha=0.5, ls="dotted", clip_on=False)
    ax1.text(size-0.25, ytop, label, color='k', alpha=0.5, va="top", ha="right")

# SET BOUNDARIES AND SAVE FIGURE -----------------------------------------------
plt.tight_layout()
fig.subplots_adjust(hspace=0.1)
pp = pdf.PdfPages(savename, keep_empty=False)
pp.savefig(fig)
pp.close()
plt.show()